# 1) Import packages

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from Code.utility import get_data, get_metrics
import warnings
from sklearn import tree
warnings.filterwarnings('ignore')

# 2) Load Dataset
Divide in train-test-validation set

In [10]:
dataset_used = "adult"
attr = "sex"
dataset_orig, privileged_groups,unprivileged_groups,optim_options = get_data(dataset_used, attr)
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
dataset_orig_test,dataset_orig_valid = dataset_orig_test.split([0.5], shuffle=True)

# Copies are needed later on to determine fairness and accuracy
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)

# 3) Train Default classification model

In [11]:
clf = LogisticRegression()
#clf = tree.DecisionTreeClassifier(random_state=1)
clf = clf.fit(dataset_orig_train.features, dataset_orig_train.labels)

## 3.1) Determine performance on the test set

In [12]:
acc,stat,aod,eod = get_metrics(clf,dataset_orig_test,dataset_orig_test_pred,unprivileged_groups,privileged_groups)

In [13]:
print ("Performance of classification model before post-processing")
print ("Accuracy:",acc)
print ("Statistical Parity:",stat)
print ("Average Odds Difference:",aod)
print ("Equal Opportunity Difference:",eod)

Performance of classification model before post-processing
Accuracy: 0.8375350140056023
Statistical Parity: 0.1884282294232028
Average Odds Difference: 0.11315377465774162
Equal Opportunity Difference: 0.13943344265924912


# Apply optimization

In [14]:
from Code.optimize import optimize_dt,optimize_lr
# Optimze for accuracy and fairness metric 0 = SPD; 1 = AOD, 2 = EOD.
#optimized_clf = optimize_dt(clf, dataset_orig_valid, dataset_orig_valid_pred,unprivileged_groups,privileged_groups,operations = 2500, fairness_metric = 1)
optimized_clf = optimize_lr(clf, dataset_orig_valid, dataset_orig_valid_pred,unprivileged_groups,privileged_groups,operations = 2500, fairness_metric = 0)

In [15]:
acc,stat,aod,eod = get_metrics(optimized_clf,dataset_orig_test,dataset_orig_test_pred,unprivileged_groups,privileged_groups)
print ("Performance of classification model after post-processing")
print ("Accuracy:",acc)
print ("Statistical Parity:",stat)
print ("Average Odds Difference:",aod)
print ("Equal Opportunity Difference:",eod)

Performance of classification model after post-processing
Accuracy: 0.83812472357364
Statistical Parity: 0.17173730173609442
Average Odds Difference: 0.10452686563725203
Equal Opportunity Difference: 0.13734656315301474
